In [1]:
import torch
import numpy as np
import os
import torchvision
import crypten
import sklearn
import matplotlib.pyplot as plt
from torchvision import transforms
import tqdm
import crypten.mpc as mpc
import crypten.communicator as comm

/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crypten.init()

ALICE = 0
BOB = 1

In [6]:
subsample = 0.02
            
@mpc.run_multiprocess(world_size=2)
def train():
    """Apply data labeling access control model"""
    # Alice loads features, Bob loads labels
    data_root_dir = './poisoned_data_samples/one_hot_target/data_samples_sub50x'
    data_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_0.pth'), src=ALICE)
    targets_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_0.pth'), src=ALICE)
    
    data_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_1.pth'), src=BOB)
    targets_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_1.pth'), src=BOB)
    
    val_data_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_0.pth'), src=ALICE)
    val_targets_alice_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_0.pth'), src=ALICE)
    
    val_data_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/data_1.pth'), src=BOB)
    val_targets_bob_enc = crypten.load_from_party(os.path.join(data_root_dir, '2/train/targets_1.pth'), src=BOB)
    
    model = torchvision.models.resnet50(num_classes = 10)
    model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    all_data = crypten.cat([data_alice_enc, data_bob_enc], dim=0)
    all_targets = crypten.cat([targets_alice_enc, targets_bob_enc], dim=0)
    
    all_val_data = crypten.cat([val_data_alice_enc, val_data_bob_enc], dim=0)
    all_val_targets = crypten.cat([val_targets_alice_enc, val_targets_bob_enc], dim=0)
    
    
    dummy_input = torch.empty(8, 1, 28, 28)
    resnet_plaintext = torchvision.models.resnet18(num_classes = 10)
    resnet_plaintext.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model = crypten.nn.from_pytorch(resnet_plaintext, dummy_input)
    model.encrypt()
    
    model.train()
    
    loss = crypten.nn.MSELoss()
    
    lr = 0.001
    num_epochs = 2
    batch_size = 8
    
    num_batches = all_data.size(0) // batch_size
    
    batch_losses = []
    
    for i in range(num_epochs):
        crypten.print(f"Epoch {i}")
        for batch in range(num_batches):
            
            start, end = batch * batch_size, (batch + 1) * batch_size
                                    
            # construct CrypTensors out of training examples / labels
            x_train = all_data[start:end]
            y_train = all_targets[start:end]
            #y_train = crypten.cryptensor(y_batch, requires_grad=True)
            
            # perform forward pass:
            output = model(x_train)
            loss_value = loss(output, y_train)
            
            # set gradients to "zero" 
            model.zero_grad()

            # perform backward pass: 
            loss_value.backward()

            # update parameters
            model.update_parameters(lr)
            
            # Print progress every batch:
            batch_loss = loss_value.get_plain_text().detach()
            batch_losses.append(batch_loss)
            crypten.print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")
            
    np.save('batch_losses_poisoned.npy', batch_losses)


In [7]:
train()

/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9

Epoch 0


/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/future/u/alexder/anaconda3/envs/cs356v2/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or

	Batch 1 of 150 Loss 0.4636
	Batch 2 of 150 Loss 0.2796
	Batch 3 of 150 Loss 0.2518
	Batch 4 of 150 Loss 0.2974
	Batch 5 of 150 Loss 0.2151
	Batch 6 of 150 Loss 0.1841
	Batch 7 of 150 Loss 0.1503
	Batch 8 of 150 Loss 0.1087
	Batch 9 of 150 Loss 0.1299
	Batch 10 of 150 Loss 0.0957
	Batch 11 of 150 Loss 0.1197
	Batch 12 of 150 Loss 0.1255
	Batch 13 of 150 Loss 0.1273
	Batch 14 of 150 Loss 0.0877
	Batch 15 of 150 Loss 0.1204
	Batch 16 of 150 Loss 0.1036
	Batch 17 of 150 Loss 0.1134
	Batch 18 of 150 Loss 0.1094
	Batch 19 of 150 Loss 0.1013
	Batch 20 of 150 Loss 0.1041
	Batch 21 of 150 Loss 0.1100
	Batch 22 of 150 Loss 0.1022
	Batch 23 of 150 Loss 0.1122
	Batch 24 of 150 Loss 0.0929
	Batch 25 of 150 Loss 0.1027
	Batch 26 of 150 Loss 0.1104
	Batch 27 of 150 Loss 0.1040
	Batch 28 of 150 Loss 0.0814
	Batch 29 of 150 Loss 0.1071
	Batch 30 of 150 Loss 0.0907
	Batch 31 of 150 Loss 0.1019
	Batch 32 of 150 Loss 0.1039
	Batch 33 of 150 Loss 0.1141
	Batch 34 of 150 Loss 0.1024
	Batch 35 of 150 Loss 0

[None, None]